# Spherical Tensors: 
## *Decomposing a matrix into irreducible spherical components*

In [1]:
import numpy as np

In this notebook, we'll generate a 3x3 matrix, decompose it, and demonstrate the rotational behavior of the components

In [12]:
Ai=np.random.rand(3,3)
print(Ai)

[[0.70496514 0.307791   0.9967534 ]
 [0.81643287 0.88261994 0.24257709]
 [0.96673373 0.18123533 0.79048781]]


The matrix functions for rotation about the *y* and *z* axes are given as follows:


\begin{equation}
\mathbf{R}_z(\theta)=\begin{pmatrix}
\cos\theta & -\sin\theta & 0 \\
\sin\theta & \cos\theta & 0 \\
0 & 0 & 1
\end{pmatrix}
\end{equation}
\begin{equation}
\mathbf{R}_y(\theta)=\begin{pmatrix}
\cos\theta & 0 & \sin\theta \\
0 & 1 & 0 \\
-\sin\theta & 0 & \cos\theta
\end{pmatrix}
\end{equation}
A arbitrary rotation of a tensor may be expressed as a product of a sequence of 3 rotations, first about *z*, then about *y*, and finally about *z* again. This is given by:
\begin{equation}
\mathbf{R}_{zyz}(\alpha,\beta,\gamma)=\mathbf{R}_z(\gamma)\cdot\mathbf{R}_y(\beta)\cdot\mathbf{R}_z(\alpha)
\end{equation}
Rotation of the matrix, $\mathbf{A}^i$ to its final value, $\mathbf{A}^f$, is then given by
\begin{equation}
\mathbf{A}^f=\mathbf{R}_{zyz}(\alpha,\beta,\gamma)\cdot\mathbf{A}^i\mathbf{R}_{zyz}^{-1}
\end{equation}
Note that
\begin{equation}
\mathbf{R}_{zyz}^{-1}=\mathbf{R}_z(-\gamma)\cdot\mathbf{R}_y(-\beta)\cdot\mathbf{R}_z(-\alpha)=\mathbf{R}_{zyz}(-\gamma,-\beta,-\alpha)=\mathbf{R}_{zyz}^\dagger(\alpha,\beta,\gamma)
\end{equation}
The code for generating these matrices (here as *lambda* functions) is given below

In [16]:
Rz=lambda theta:np.array([[np.cos(theta),-np.sin(theta),0],
                          [np.sin(theta), np.cos(theta),0],
                          [0,             0,            1]])
Ry=lambda theta:np.array([[np.cos(theta), 0,np.sin(theta)],
                          [0,             1,            0],
                          [-np.sin(theta),0,np.cos(theta)]])
Rzyz=lambda alpha,beta,gamma:Rz(gamma)@Ry(beta)@Rz(alpha)    #Use the @ symbol for matrix multplication!!
Rzyzi=lambda alpha,beta,gamma:Rzyz(alpha,beta,gamma).T

So, what happens when we rotate the matrix $\mathbf{A}$ by some random euler angles?

In [17]:
alpha,beta,gamma=np.random.rand(3)*np.pi  #Random set of euler angles (don't re-run this cell to keep same angles)
print(f'alpha={alpha*180/np.pi:.1f}, beta={beta*180/np.pi:.1f}, gamma={gamma*180/np.pi:.1f}')

alpha=155.5, beta=96.6, gamma=86.6


In [19]:
Af=Rzyz(alpha,beta,gamma)@Ai@Rzyzi(alpha,beta,gamma)
print(Af)

[[ 0.41061938 -0.07669251  0.76318223]
 [-0.17813516  1.03609509  0.92798909]
 [ 0.26682283  1.00994805  0.93135841]]


Nothing really special happens. We just get a new matrix. But, what if we decompose the matrix $\mathbf{A_i}$ into special components, that is, into an isotropic part, a symmetric part, and an anti-symmetric part. These parts are given as follows:
\begin{equation}\mathbf{A}_{iso}=\frac{trace(\mathbf{A})}{3}\begin{pmatrix}1&0&0\\0&1&0\\0&0&1\end{pmatrix}
\end{equation}
Trace is the sum of the diagonal
\begin{equation}
\mathbf{A}_{sym}=\frac12\left(A+A^\dagger\right)-\mathbf{A}_{iso}
\end{equation}
\begin{equation}
\mathbf{A}_{anti}=\frac12\left(A-A^\dagger\right)
\end{equation}
These functions are coded below

In [25]:
Iso=lambda A:np.trace(A)/3*np.eye(3)
Sym=lambda A:0.5*(A+A.T)-Iso(A)
Anti=lambda A:0.5*(A-A.T)

Then, we may decompose the matrix $\mathbf{A}^i$ into components $\mathbf{A}_{iso}$, $\mathbf{A}_{sym}$, and $\mathbf{A}_{anti}$ and furthermore verify that they sum up to $\mathbf{A}^i$

In [260]:
Aiso_i=Iso(Ai)
Asym_i=Sym(Ai)
Aanti_i=Anti(Ai)
print(f'Aiso_i=\n{Aiso_i}')
print(f'Asym_i=\n{Asym_i}')
print(f'Aanti_i=\n{Aanti_i}')
print(f'Aiso_i+Asym_i+Aanti_i-Ai\n{Aiso_i+Asym_i+Aanti_i-Ai}')

Aiso_i=
[[0.79269096 0.         0.        ]
 [0.         0.79269096 0.        ]
 [0.         0.         0.79269096]]
Asym_i=
[[-0.08772582  0.56211194  0.98174356]
 [ 0.56211194  0.08992898  0.21190621]
 [ 0.98174356  0.21190621 -0.00220315]]
Aanti_i=
[[ 0.         -0.25432093  0.01500983]
 [ 0.25432093  0.          0.03067088]
 [-0.01500983 -0.03067088  0.        ]]
Aiso_i+Asym_i+Aanti_i-Ai
[[ 0.00000000e+00 -5.55111512e-17  0.00000000e+00]
 [-1.11022302e-16  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]


We see, then that $\mathbf{A}_{iso}$ just has diagonal elements which are all equal, $\mathbf{A}_{sym}$ has a diagonal that sums to zero, and the off-diagonal elements are symmetric, and $\mathbf{A}_{anti}$ has no diagonal, and off-diagonal elements that are anti-symmetric. 

What is furthermore interesting is the behavior of these components under rotation. We apply the euler angles ($\alpha$,$\beta$,$\gamma$) to each of the components, and furthermore sum the answer to verify that they yield $\mathbf{A}^f$

In [261]:
Aiso_f=Rzyz(alpha,beta,gamma)@Aiso_i@Rzyzi(alpha,beta,gamma)
Asym_f=Rzyz(alpha,beta,gamma)@Asym_i@Rzyzi(alpha,beta,gamma)
Aanti_f=Rzyz(alpha,beta,gamma)@Aanti_i@Rzyzi(alpha,beta,gamma)
print(f'Aiso_f=\n{Aiso_f}')
print(f'Asym_f=\n{Asym_f}')
print(f'Aanti_f=\n{Aanti_f}')
print(f'Aiso_f+Asym_f+Aanti_f=\n{Aiso_i+Asym_i+Aanti_i}')
print(f'Aiso_f+Asym_f+Aanti_f-Af\n{Aiso_i+Asym_i+Aanti_i-Ai}')

Aiso_f=
[[ 7.92690962e-01 -4.92292359e-18 -2.33109100e-18]
 [-6.19982856e-18  7.92690962e-01  6.29931433e-18]
 [-1.75250656e-18  1.36467340e-17  7.92690962e-01]]
Asym_f=
[[-0.38207158 -0.12741384  0.51500253]
 [-0.12741384  0.24340413  0.96896857]
 [ 0.51500253  0.96896857  0.13866745]]
Aanti_f=
[[-6.39735875e-18  5.07213247e-02  2.48179703e-01]
 [-5.07213247e-02 -1.29278646e-19 -4.09794837e-02]
 [-2.48179703e-01  4.09794837e-02 -1.29979578e-17]]
Aiso_f+Asym_f+Aanti_f=
[[0.70496514 0.307791   0.9967534 ]
 [0.81643287 0.88261994 0.24257709]
 [0.96673373 0.18123533 0.79048781]]
Aiso_f+Asym_f+Aanti_f-Af
[[ 0.00000000e+00 -5.55111512e-17  0.00000000e+00]
 [-1.11022302e-16  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]


As expected, the sum of the rotation of the individual components yields the rotation of the total matrix. However, what we should also see is that the isotropic matrix remains isotropic, the symmetric matrix remains symmetric, and the anti-symmetric matrix remains anti-symmetric.

***These components do not mix for any arbitrary rotation***

However, we can go further in decomposing the matrix. Note that the isotropic matrix is defined by a single value, the anti-symmetric matrix by 3 independent values, and the symmetric matrix is defined by 5 independent values. Then, we will decompose these matrices into 1, 3, and 5 additional matrices ($A_{iso}^0$,$A_{anti}^{-1}$,$A_{anti}^{0}$,$A_{anti}^{1}$,$A_{sym}^{-2}$,$A_{sym}^{-1}$,$A_{sym}^{0}$,$A_{sym}^{1}$,$A_{sym}^{2}$). 

The basis matrices are as follows:
\begin{equation}
\mathbf{M}_{iso}^0=\begin{pmatrix}1&0&0\\0&1&0\\0&0&1\end{pmatrix}
\end{equation}


\begin{equation}
\mathbf{M}_{anti}^0=\begin{pmatrix}0&-1&0\\1&0&0\\0&0&0\end{pmatrix},
\mathbf{M}_{anti}^{\pm1}=\frac12\begin{pmatrix}0&0&-1\\0&0&\mp i\\1&\pm i&0\end{pmatrix}
\end{equation}


\begin{equation}
\mathbf{M}_{sym}^0=\frac12\begin{pmatrix}-1&0&0\\0&-1&0\\0&0&2\end{pmatrix},
\mathbf{M}_{sym}^{\pm1}=\frac12\begin{pmatrix}0&0&\pm1\\0&0&i\\\pm1&i&0\end{pmatrix},
\mathbf{M}_{sym}^{\pm2}=\frac12\begin{pmatrix}1&\pm i&0\\\pm i&-1&0\\0&0&0\end{pmatrix}
\end{equation}

Below, we define these matrices, and then project that matrix out of \mathbf{A}_i

In [292]:
Iso_0=lambda A:Iso(A)  #This one doesn't get change

M={'iso':{},'anti':{},'sym':{}}  #Storage for all the matrices
M['iso'][0]=-1/np.sqrt(3)*np.eye(3)

M['anti'][-1]=-0.5*np.array([[0,0,-1],[0,0,-1j],[1,1j,0]])
M['anti'][0]=-1j/np.sqrt(2)*np.array([[0,-1,0],[1,0,0],[0,0,0]])
M['anti'][1]=-0.5*np.array([[0,0,-1],[0,0,1j],[1,-1j,0]])

M['sym'][-2]=0.5*np.array([[1,-1j,0],[-1j,-1,0],[0,0,0]])
M['sym'][-1]=0.5*np.array([[0,0,-1],[0,0,1j],[-1,1j,0]])
M['sym'][0]=1/np.sqrt(6)*np.diag([-1,-1,2])
M['sym'][1]=0.5*np.array([[0,0,1],[0,0,1j],[1,1j,0]])
M['sym'][2]=0.5*np.array([[1,1j,0],[1j,-1,0],[0,0,0]])


#First, a general function for selecting out a component of the matrix
CompA=lambda M,A:np.trace(A@M.conj().T)/np.trace(M@M.conj().T)
CompMat=lambda M,A:CompA(M,A)*M


funsA={key:{k:(lambda A,v=v:CompA(v,A)) for k,v in value.items()} for key,value in M.items()}
funs={key:{k:(lambda A,v=v:CompMat(v,A)) for k,v in value.items()} for key,value in M.items()}

In [293]:
for key,value in funs.items():
    print(f'A{key} components:')
    for fun in value.values():
        Ac=fun(Ai)
        out='[['
        for k,Ac0 in enumerate(Ac):
            out+=','.join([f'{Ac00:12.3f}' for Ac00 in Ac0])+']'
            out+=']\n' if k==2 else '\n ['
        print(out)
        

Aiso components:
[[       0.793,       0.000,       0.000]
 [       0.000,       0.793,       0.000]
 [       0.000,       0.000,       0.793]]

Aanti components:
[[0.000+0.000j,0.000+0.000j,0.008-0.015j]
 [0.000+0.000j,0.000+0.000j,0.015+0.008j]
 [-0.008+0.015j,-0.015-0.008j,0.000+0.000j]]

[[0.000+0.000j,-0.254+0.000j,0.000+0.000j]
 [0.254+0.000j,0.000+0.000j,0.000+0.000j]
 [0.000+0.000j,0.000+0.000j,0.000+0.000j]]

[[-0.000+0.000j,-0.000+0.000j,0.008+0.015j]
 [-0.000+0.000j,-0.000+0.000j,0.015-0.008j]
 [-0.008-0.015j,-0.015+0.008j,-0.000+0.000j]]

Asym components:
[[-0.044+0.281j,0.281+0.044j,-0.000+0.000j]
 [0.281+0.044j,0.044-0.281j,-0.000+0.000j]
 [-0.000+0.000j,-0.000+0.000j,-0.000+0.000j]]

[[0.000-0.000j,0.000-0.000j,0.491+0.106j]
 [0.000-0.000j,0.000-0.000j,0.106-0.491j]
 [0.491+0.106j,0.106-0.491j,0.000-0.000j]]

[[       0.001,      -0.000,      -0.000]
 [      -0.000,       0.001,      -0.000]
 [      -0.000,      -0.000,      -0.002]]

[[0.000+0.000j,0.000+0.000j,0.491-0.

As a quick check, we sum together all components to verify that they are equal to the original matrix

In [294]:
Ai_check=np.sum([np.sum([f(Ai) for f in fun.values()],axis=0) for fun in funs.values()],axis=0)
print(Ai-Ai_check)

[[-1.11022302e-16+0.j  1.11022302e-16+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j -1.11022302e-16+0.j  0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]]


Now that we have the decomposition, we investigate their behavior under rotation about the *z*-axis. We'll rotate by some random angle, $\phi$, and show that for each component, the value after rotation is $\mathbf{A}_{\zeta}^{m}=e^{-i\phi m}\mathbf{A}_{\zeta}^{m}$. We'll just run the rotation directly and via multiplication.

In [295]:
phi=np.random.rand()*2*np.pi
print(f'phi = {phi*180/np.pi:.1f} degrees')

phi = 292.9 degrees


In [296]:
for key,value in funs.items():
    print(f'A{key} components:')
    for m,fun in value.items():
        Arot_dir=Rz(phi)@fun(Ai)@Rz(-phi)
        Arot_mul=fun(Ai)*np.exp(-1j*phi*m)
        out='[['
        for k,(Ad0,Am0) in enumerate(zip(Arot_dir,Arot_mul)):
            out+=','.join([f'{A00:15.3f}' for A00 in Ad0])+']'
            out+=']\t' if k==2 else '\t ['
            
            out+=','.join([f'{A00:15.3f}' for A00 in Am0])+']'
            out+=']\n' if k==2 else '\n ['
        print(out)

Aiso components:
[[          0.793,          0.000,          0.000]	 [   0.793+0.000j,   0.000+0.000j,   0.000+0.000j]
 [          0.000,          0.793,          0.000]	 [   0.000+0.000j,   0.793+0.000j,   0.000+0.000j]
 [          0.000,          0.000,          0.793]]	   0.000+0.000j,   0.000+0.000j,   0.793+0.000j]]

Aanti components:
[[   0.000+0.000j,   0.000+0.000j,   0.017+0.001j]	 [   0.000+0.000j,   0.000+0.000j,  -0.011-0.013j]
 [   0.000+0.000j,   0.000+0.000j,  -0.001+0.017j]	 [   0.000+0.000j,   0.000+0.000j,   0.013-0.011j]
 [  -0.017-0.001j,   0.001-0.017j,   0.000+0.000j]]	   0.011+0.013j,  -0.013+0.011j,   0.000+0.000j]]

[[  -0.000+0.000j,  -0.254+0.000j,   0.000+0.000j]	 [   0.000+0.000j,  -0.254+0.000j,   0.000+0.000j]
 [   0.254+0.000j,  -0.000+0.000j,   0.000+0.000j]	 [   0.254+0.000j,   0.000+0.000j,   0.000+0.000j]
 [   0.000+0.000j,   0.000+0.000j,   0.000+0.000j]]	   0.000+0.000j,   0.000+0.000j,   0.000+0.000j]]

[[   0.000+0.000j,   0.000+0.000j,   0.017-0

Indeed, the rotation of the individual components about the *z*-axis can be expressed as a simple multiplication by $e^{im\phi}$

## Working with Spherical Tensor Components

Rather than carrying around 9 different matrices to rotate our components, we instead simply use the coefficient of each matrix, so that we only require these 9 values. Note that rather than call the parts "isotropic", "antisymmetric" and "symmetric", we often call them the rank-0, rank-1, and rank-2 parts, where the rank refers to the maximum ratio of modulation to the rotation rate.

In [297]:
A={key:{m:f(Ai) for m,f in fun0.items()} for key,fun0 in funsA.items()}
for key,value in A.items():
    print(f'\n{key}:\n')
    for k,v in value.items():
        print(f'{k}\t:\t{v:.3f}\t')


iso:

0	:	-1.373	

anti:

-1	:	0.015-0.031j	
0	:	0.000+0.360j	
1	:	0.015+0.031j	

sym:

-2	:	-0.089+0.562j	
-1	:	-0.982-0.212j	
0	:	-0.003	
1	:	0.982-0.212j	
2	:	-0.089-0.562j	


The above values are extracted by projecting the matrices onto $\mathbf{A}$. However, we can also get them based on the following definitions, where \mathbf{A} is given as follows:
\begin{equation}
\mathbf{A}=\begin{pmatrix}a_{xx}&a_{xy}&a_{xz}\\a_{yx}&a_{yy}&a_{yz}\\a_{zx}&a_{zy}&a_{zz}\end{pmatrix}
\end{equation}
Then, the amplitudes of the spherical components are given by
\begin{eqnarray}
A_{0,0}&=&-\frac{1}{\sqrt{3}}(a_{xx}+a_{yy}+a_{zz}) \\
A_{1,0}&=&-\frac{i}{\sqrt{2}}(a_{xy}-a_{yx}) \\
A_{1,\pm1}&=&-\frac12(a_{zx}-a_{xz}\pm i(a_{zy}-a_{yz})) \\
A_{2,0}&=&\frac{1}{\sqrt{6}}(3a_{zz}-(a_{xx}+a_{yy}+a_{zz}) \\
A_{2,\pm1}&=&\mp\frac12(a_{xz}+a_{zx}\pm i(a_{yz}+a_{zy})) \\
A_{2,\pm2}&=&\frac12(a_{xx}-a_{yy}\pm i(a_{xy}+a_{yx})
\end{eqnarray}

Then, we can check our values from above:

In [301]:
print(f'A_(0,0)={-1/np.sqrt(3)*(Ai[0,0]+Ai[1,1]+Ai[2,2]):.3f}\n')
print(f'A_(1,-1)={-0.5*(Ai[2,0]-Ai[0,2]-1j*(Ai[2,1]-Ai[1,2])):.3f}')
print(f'A_(1,0)={-1j/np.sqrt(2)*(Ai[0,1]-Ai[1,0]):.3f}')
print(f'A_(1,1)={-0.5*(Ai[2,0]-Ai[0,2]+1j*(Ai[2,1]-Ai[1,2])):.3f}\n')
print(f'A_(2,-2)={0.5*(Ai[0,0]-Ai[1,1]-1j*(Ai[0,1]+Ai[1,0])):.3f}')
print(f'A_(2,-1)={-0.5*(Ai[0,2]+Ai[2,0]-1j*(Ai[1,2]+Ai[2,1])):.3f}')
print(f'A_(2,0)={1/np.sqrt(6)*(3*Ai[2,2]-(Ai[0,0]+Ai[1,1]+Ai[2,2])):.3f}')
print(f'A_(2,1)={-0.5*(Ai[0,2]+Ai[2,0]-1j*(Ai[1,2]+Ai[2,1])):.3f}')
print(f'A_(2,2)={0.5*(Ai[0,0]-Ai[1,1]+1j*(Ai[0,1]+Ai[1,0])):.3f}')

A_(0,0)=-1.373

A_(1,-1)=0.015-0.031j
A_(1,0)=0.000+0.360j
A_(1,1)=0.015+0.031j

A_(2,-2)=-0.089-0.562j
A_(2,-1)=-0.982+0.212j
A_(2,0)=-0.003
A_(2,1)=-0.982+0.212j
A_(2,2)=-0.089+0.562j


## Rotational components of CSA under MAS
Suppose we have a CSA with anisotropy, $\delta$, asymmetry, $\eta$, euler angles (PAS to rotor frame) ($\alpha$,$\beta$,$\gamma$) and isotropic value, $\overline{a}$. What are the rotating components, $A_n$, from which we can construct the CSA Hamiltonian in the rotating frame (high-field approximation):
\begin{equation}
H_{CSA}=\sum\limits_{l=0}^2{\sum\limits_{n=-2}^2{e^{-in\omega_rt}A_n^{l}\mathscr{T}_{l,0}}}
\end{equation}

In [308]:
delta=100  #ppm
eta=.4
iso=180    #ppm
alpha,beta,gamma=np.random.rand(3)*np.pi

First, we need to get the spherical tensor in its principal axis system (PAS). For this, we use the following formulas:
\begin{eqnarray}
A_{0,0}&=&-\sqrt{3}\overline{a} \\
A_{2,0}&=&\sqrt{\frac32}\delta \\
A_{2,\pm1}&=&0 \\
A_{2,\pm2}&=&-\frac12\delta\eta
\end{eqnarray}

In [312]:
A00=-np.sqrt(3)*iso
A2PAS=np.array([-0.5*delta*eta,0,np.sqrt(3/2)*delta,0,-0.5*delta*eta],dtype=complex)  #Store in a complex array
print(A00)
print(A2PAS)

-311.7691453623979
[-20.        +0.j   0.        +0.j 122.47448714+0.j   0.        +0.j
 -20.        +0.j]


Next, we need to rotate the A2 terms into the rotor frame by the Euler angles. For this, we need the functions $d_{m',m}^2(\beta)$ and $D_{m',m}^2(\alpha,\beta,\gamma)$ in our code

In [313]:
def d2(beta,mp,m):
    c,s=np.cos(beta),np.sin(beta)
    if mp==-2:
        if m==-2:x=0.25*(1+c)**2
        if m==-1:x=0.5*(1+c)*s
        if m==0:x=np.sqrt(3/8)*s**2
        if m==1:x=0.5*(1-c)*s
        if m==2:x=0.25*(1-c)**2
    if mp==-1:
        if m==-2:x=-0.5*(1+c)*s
        if m==-1:x=c**2-0.5*(1-c)
        if m==0:x=np.sqrt(3/8)*2*c*s
        if m==1:x=0.5*(1+c)-c**2
        if m==2:x=0.5*(1-c)*s
    if mp==0:
        if m==-2:x=np.sqrt(3/8)*s**2
        if m==-1:x=-np.sqrt(3/8)*2*s*c
        if m==0:x=0.5*(3*c**2-1)
        if m==1:x=np.sqrt(3/8)*2*s*c
        if m==2:x=np.sqrt(3/8)*s**2
    if mp==1:
        if m==-2:x=-0.5*(1-c)*s
        if m==-1:x=0.5*(1+c)-c**2
        if m==0:x=-np.sqrt(3/8)*2*s*c
        if m==1:x=c**2-0.5*(1-c)
        if m==2:x=0.5*(1+c)*s
    if mp==2:
        if m==-2:x=0.25*(1-c)**2
        if m==-1:x=-0.5*(1-c)*s
        if m==0:x=np.sqrt(3/8)*s**2
        if m==1:x=-0.5*(1+c)*s
        if m==2:x=0.25*(1+c)**2
    return x

Recall that $D^2_{m',m}(\alpha,\beta,\gamma)=e^{-im\gamma}d^2_{m',m}(\beta)e^{-im'\alpha}$

In [314]:
D2=lambda alpha,beta,gamma,mp,m:np.exp(-1j*m*gamma)*d2(beta,mp,m)*np.exp(-1j*mp*alpha)

For each term in the rotor frame, we need to sum over all components of the tensor in the PAS ($m'$) multiplied by $D_{m',m}^2$

In [317]:
A2RF=np.zeros(5,dtype=complex)  #Storage for components in the rotor frame
for m in range(-2,3):   #Loop over all final components, m
    for mp in range(-2,3):  #Loop over all initial components mp
        A2RF[m+2]+=D2(alpha,beta,gamma,mp,m)*A2PAS[mp+2]   #Cumulative sum over contributions from 5 components
print(A2RF)

[-69.6976387 -14.4309062j  -15.94210752-42.47334659j
 -39.39167786 +0.j          15.94210752-42.47334659j
 -69.6976387 +14.4309062j ]


Then, we need the components in the lab frame, which requires rotation about the *y*-axis by -54.74$^\circ$, i.e. the magic angle. Since we will assume the high-field approximation, we only calculate the $A_{2,0}^\mathrm{LAB}$  component (set $m=0$)

In [319]:
A20LAB=0
phi_m=np.arccos(np.sqrt(1/3))
for mp in range(-2,3): #Loop over all initial components
    A20LAB+=d2(-phi_m,mp=mp,m=0)*A2RF[mp+2]
print(A20LAB)

(-38.499523561882086+4.440892098500626e-15j)


This term goes with the rank-2 spin-part of the spherical tensor, whereas the isotropic term, $A_{0,0}$ goes with the rank-0 spin-part of the spherical tensor, such that the Hamiltonian is:
\begin{equation}
H_{CSA}=A_{0,0}\left(-\frac{1}{\sqrt{3}}I_z\right)+\sum\limits_{n=-2}^2{e^{-in\omega_rt}A_n\left(\sqrt{\frac23}I_z\right)}
\end{equation}

For simpler notation, one may divide $A_{0,0}$ by $-\sqrt{2}$ and add to to $A_0$, allowing us to drop the first term above